# Causal impact (counterfactual prediction) on a time series

This notebook demonstrates an **impact-like** estimator:

- train a model on the pre-period (`y ~ X`)
- forecast the counterfactual for the post-period
- estimate point / cumulative / relative effect
- uncertainty via block bootstrap
- diagnostics and placebo-in-time guardrail


In [ ]:
import sys
from pathlib import Path

# Ensure `src/` is importable when running from repo root
repo_root = Path.cwd()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tecore.causal import DataSpec, ImpactConfig, ImpactMethod, run_impact


## 1) Load data

In [ ]:
df = pd.read_csv(repo_root / "data" / "example_ts" / "example_daily.csv")
df.head()

## 2) Configure the run

Set:
- intervention date
- covariates
- bootstrap budget


In [ ]:
spec = DataSpec(
    date_col="date",
    y_col="y",
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    freq="D",
    missing_policy="raise",
    aggregation="mean",
    add_time_trend=True,
    add_day_of_week=True,
)

cfg = ImpactConfig(
    intervention_date="2025-05-01",
    method=ImpactMethod.CAUSAL_IMPACT_LIKE,
    ridge_alpha=1.0,
    bootstrap_iters=200,
    block_size=7,
    alpha=0.05,
    run_placebo=True,
    n_placebos=25,
)

res = run_impact(df, spec, cfg)
res.summary()

## 3) Effect series

In [ ]:
eff = res.effect_series.copy()
eff["date"] = pd.to_datetime(eff["date"])
eff.head()

### Observed vs counterfactual

In [ ]:
plt.figure()
plt.plot(eff["date"], eff["y"], label="observed")
plt.plot(eff["date"], eff["y_hat"], label="counterfactual")
plt.axvline(pd.Timestamp(cfg.intervention_date), linestyle="--")
plt.title("Observed vs counterfactual")
plt.legend()
plt.tight_layout()
plt.show()

### Point effect (with CI)

In [ ]:
plt.figure()
plt.plot(eff["date"], eff["point_effect"], label="point_effect")
plt.fill_between(eff["date"], eff["point_ci_low"], eff["point_ci_high"], alpha=0.2, label="CI")
plt.axhline(0.0, linestyle="--")
plt.axvline(pd.Timestamp(cfg.intervention_date), linestyle="--")
plt.title("Point effect (daily)")
plt.legend()
plt.tight_layout()
plt.show()

### Cumulative effect (with CI)

In [ ]:
plt.figure()
plt.plot(eff["date"], eff["cum_effect"], label="cum_effect")
plt.fill_between(eff["date"], eff["cum_ci_low"], eff["cum_ci_high"], alpha=0.2, label="CI")
plt.axhline(0.0, linestyle="--")
plt.axvline(pd.Timestamp(cfg.intervention_date), linestyle="--")
plt.title("Cumulative effect")
plt.legend()
plt.tight_layout()
plt.show()

## 4) Diagnostics and warnings

In [ ]:
print("Warnings:", res.warnings)
print("Diagnostics:", res.diagnostics)

## 5) Placebo-in-time guardrail

In [ ]:
if res.placebo_results is None:
    print("Placebo was not run.")
else:
    pl = res.placebo_results.copy()
    pl["placebo_date"] = pd.to_datetime(pl["placebo_date"])
    pl.head()

In [ ]:
if res.placebo_results is not None:
    plt.figure()
    plt.hist(res.placebo_results["cum_effect"].values, bins=15)
    plt.axvline(res.cum_effect, linestyle="--")
    plt.title("Placebo distribution of cumulative effect")
    plt.tight_layout()
    plt.show()

    print("Estimated cumulative effect:", res.cum_effect)
    print("Placebo p-value:", res.p_value)

## 6) Save outputs

In [ ]:
out_dir = repo_root / "out" / "notebook_09_causal_impact"
out_dir.mkdir(parents=True, exist_ok=True)

eff_out = out_dir / "effect_series.csv"
eff_to_save = res.effect_series.copy()
eff_to_save["date"] = pd.to_datetime(eff_to_save["date"]).dt.strftime("%Y-%m-%d")
eff_to_save.to_csv(eff_out, index=False)

import json
with open(out_dir / "summary.json", "w", encoding="utf-8") as f:
    json.dump(res.summary(), f, indent=2, ensure_ascii=False)

if res.placebo_results is not None:
    pl_out = out_dir / "placebo_results.csv"
    pl_to_save = res.placebo_results.copy()
    pl_to_save["placebo_date"] = pd.to_datetime(pl_to_save["placebo_date"]).dt.strftime("%Y-%m-%d")
    pl_to_save.to_csv(pl_out, index=False)

print("Wrote:", out_dir)